# Plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyabc2.sources import load_example

In [ ]:
tune = load_example("For the Love of Music")
tune

Something simple we can do is plot the trajectory, as a sort of time series.
Ignoring note duration, that looks like this:

In [ ]:
y = np.array([n.value for n in tune.iter_notes()])
x = np.arange(len(y)) * 1/8

plt.figure(figsize=(7, 3), layout="constrained")
plt.axis("off")
plt.plot(x, y);

Or, considering duration:

In [ ]:
plt.figure(figsize=(7, 3), layout="constrained")

y = np.array([n.value for n in tune.iter_notes()])
x = np.arange(1, len(y) + 1) * 1/8  # shift for consistency

plt.plot(x, y, label="ignored")

data = np.array(
    [
        [n.value, float(n.duration)]
        for n in tune.iter_notes()
    ]
)
x = data[:,1].cumsum()  # ends of notes
y = data[:,0]
assert len(x) == len(y)

plt.plot(x, y, label="considered")
plt.axis("off")
plt.legend(title="duration", loc="upper left");

The divergence occurs due to the 16th notes in the B part.

We can make a histogram of the notes, again considering duration or not.

In [ ]:
data = [
    [n.value, float(n.duration), n.to_pitch().unicode()]
    for n in tune.iter_notes()
]

df = pd.DataFrame(data, columns=["value", "duration", "pitch"])

_, ax = plt.subplots(figsize=(6, 3.5), layout="constrained")

count = (
    df.groupby("value")
    .aggregate({"pitch": "first", "duration": "size"})
    .rename(columns={"duration": "unweighted"})
    .assign(
        weighted=(
            df.assign(w=df["duration"] * 8)
            .groupby("value")["w"].sum()
        )
    )
)

count.plot.bar(
    x="pitch",
    rot=0,
    xlabel="Pitch",
    ylabel="Count",
    ax=ax,
);